In [33]:
import pandas as pd
df = pd.read_csv("""C:\\Users\\bless\\Downloads\\BD\\BD\\diabetic_data.csv""")
list(df['age'].unique())
for i in list(df['age'].unique()):
    x = str(i)
    x =x[1:]
    x =x[:-1]
    agerange = x.split("-")
    age = int((int(agerange[1])+int(agerange[0]))/2)
    df["age"] = df["age"].replace([i],age)
df.drop(df[df['age'] <= 50].index,inplace = True)
df['age'].value_counts()



75    26068
65    22483
55    17256
85    17197
95     2793
Name: age, dtype: int64

In [25]:
import pandas as pd
df = pd.read_csv("""C:\\Users\\bless\\Downloads\\BD\\BD\\diabetic_data.csv""", na_values=["?"])
data = [df['num_medications'], df['number_outpatient'], df['number_emergency'], df['time_in_hospital'],
df['number_inpatient'], df['encounter_id'], df['age'], df['num_lab_procedures'], df['number_diagnoses'],
df['num_procedures'], df['readmitted']]
dfm = pd.concat(data, axis = 1)
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   num_medications     101766 non-null  int64 
 1   number_outpatient   101766 non-null  int64 
 2   number_emergency    101766 non-null  int64 
 3   time_in_hospital    101766 non-null  int64 
 4   number_inpatient    101766 non-null  int64 
 5   encounter_id        101766 non-null  int64 
 6   age                 101766 non-null  object
 7   num_lab_procedures  101766 non-null  int64 
 8   number_diagnoses    101766 non-null  int64 
 9   num_procedures      101766 non-null  int64 
 10  readmitted          101766 non-null  object
dtypes: int64(9), object(2)
memory usage: 8.5+ MB


C:\Users\bless\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
import pandas as pd
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
df = pd.read_csv("""C:\\Users\\bless\\Downloads\\BD\\BD\\diabetic_data.csv""", na_values=["?"])
df.drop('weight', inplace=True, axis=1)
df['readmitted'] = df["readmitted"].replace('NO',0)
df['readmitted'] = df["readmitted"].replace('>30',1)
df['readmitted'] = df["readmitted"].replace('<30',1)
df['diabetesMed'] = df["diabetesMed"].replace('Yes',1)
df['diabetesMed'] = df["diabetesMed"].replace('No',0)
df['gender'] = df['gender'].replace('Female',0)
df['gender'] = df['gender'].replace('Male', 1)
for columns in df.columns:
    variable = df[columns].value_counts().idxmax()
    max_rv = df[columns].value_counts(normalize = True)[variable]
    if max_rv >= 0.95:
        df.drop(columns, inplace=True, axis=1)
list(df['age'].unique())
for i in list(df['age'].unique()):
    x = str(i)
    x =x[1:]
    x =x[:-1]
    agerange = x.split("-")
    age = int((int(agerange[1])+int(agerange[0]))/2)
    df["age"] = df["age"].replace([i],age)
df["diag_1"] = df["diag_1"].fillna(0)
df["diag_2"] = df["diag_2"].fillna(0)
df["diag_3"] = df["diag_3"].fillna(0)
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df['gender'] = df['gender'].astype('int64')
g = df.groupby("diag_3")
gdf = g.get_group('250')
f = gdf.groupby('readmitted')
fdf = f.get_group(1)
ndf = fdf.select_dtypes(include= "int64")
cdf = df.select_dtypes(include= 'object')
for columns in ndf.columns:
    ndf["zscore"] = (df[columns]-df[columns].mean())/df[columns].std()
    ndf = ndf[(ndf["zscore"] < 3) & (ndf["zscore"] > -3)]
    ndf.drop("zscore", inplace=True, axis=1)
ndf = ndf.drop_duplicates(subset = "patient_nbr", keep = "first")
data = [ndf['num_medications'], ndf['number_outpatient'], ndf['number_emergency'], ndf['time_in_hospital'],
ndf['number_inpatient'], ndf['encounter_id'], ndf['age'], ndf['num_lab_procedures'], ndf['number_diagnoses'],
ndf['num_procedures'], ndf['readmitted'], ndf['gender']]
dfm = pd.concat(data, axis = 1)
x = dfm.drop(columns = ['readmitted'])
y = dfm['readmitted']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
y_test


C:\Users\bless\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\bless\AppData\Local\Temp/ipykernel_21868/3052910612.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["zscore"] = (df[columns]-df[columns].mean())/df[columns].std()


100634    1
87440     1
56288     1
32611     1
85561     1
         ..
72077     1
98274     1
96614     1
40497     1
72367     1
Name: readmitted, Length: 188, dtype: int64